In [ ]:
#%pip install --upgrade -q aiobotocore
#://aws.amazon.com/tutorials/machine-learning-tutorial-build-model-locally/

In [ ]:
#%pip uninstall botocore boto3 awscli aiobotocore -y 

In [ ]:
#%pip show awscli boto3 botocore aiobotocore

In [ ]:
#%pip install aiobotocore==2.12.3 botocore boto3 awscli
# Successfully installed aiobotocore-2.12.3 awscli-1.32.69 boto3-1.34.69 botocore-1.34.69

In [ ]:
#%pip install aiobotocore==2.12.3 "botocore>=1.34.41,<1.34.70" boto3==1.34.69 awscli==1.32.69

In [ ]:
#%pip install -q  xgboost==1.3.1

In [ ]:
#%pip show xgboost

In [ ]:
#%pip show s3fs

In [ ]:
#%pip install --upgrade s3fs
#%pip install --upgrade pandas

In [ ]:
#%pip uninstall pandas -y

In [2]:
#%pip install pandas

In [ ]:
#%pip install --upgrade docutils

In [ ]:
#%pip install --upgrade sagemaker-data-insights 

In [ ]:
#%pip install --force-reinstall statsmodels==0.14.0 pydeseq2==0.4.3

In [ ]:
#%pip install antspyx  #https://github.com/ANTsX/ANTsPy/issues/508

In [ ]:
#!pip list pandas numpy scipy  numpy==1.26.4  scipy=1.13.0          
#                              numpy==1.22.4 scipy==1.7.3

In [ ]:
%pip uninstall numpy scipy antspyx scikit-learn requests statsmodels matplotlib -y

In [ ]:
%pip install  scipy==1.7.3 "numpy<1.23" "antspyx>=0.4.2" "scikit-learn" "requests" "statsmodels" "matplotlib"

In [ ]:
#%pip install scipy==1.7.3 "numpy<1.23.0,>=1.16.5" antspyx scikit-image

In [ ]:
!pip list | grep int64index

In [5]:
%pip show numpy
%pip show numpy
%pip show antspyx
%pip show scikit-image
%pip show scipy

In [ ]:
%pip uninstall pandas -y

In [ ]:
%pip install pandas

In [ ]:
%pip install --upgrade pandas
%pip install --upgrade lightgbm
%pip install dask-expr distributed dask 

In [2]:
import pandas as pd
import boto3
import sagemaker
import json
import joblib
import xgboost as xgb
from sklearn.metrics import roc_auc_score

# Set SageMaker and S3 client variables
sess = sagemaker.Session()

region = sess.boto_region_name
s3_client = boto3.client("s3", region_name=region)

sagemaker_role = sagemaker.get_execution_role()

# Set read and write S3 buckets and locations
write_bucket = sess.default_bucket()
write_prefix = "fraud-detect-demo"

read_bucket = "sagemaker-sample-files"
read_prefix = "datasets/tabular/synthetic_automobile_claims" 

train_data_key = f"{read_prefix}/train.csv"
test_data_key = f"{read_prefix}/test.csv"
model_key = f"{write_prefix}/model"
output_key = f"{write_prefix}/output"

train_data_uri = f"s3://{read_bucket}/{train_data_key}"
test_data_uri = f"s3://{read_bucket}/{test_data_key}"

hyperparams = {
                "max_depth": 3,
                "eta": 0.2,
                "objective": "binary:logistic",
                "subsample" : 0.8,
                "colsample_bytree" : 0.8,
                "min_child_weight" : 3
              }

num_boost_round = 100
nfold = 3
early_stopping_rounds = 10

# Set up data input
label_col = "fraud"
data = pd.read_csv(train_data_uri)

# Read training data and target
train_features = data.drop(label_col, axis=1)
train_label = pd.DataFrame(data[label_col])

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [3]:
train_features

,num_vehicles_involved,num_injuries,num_witnesses,police_report_available,injury_claim,vehicle_claim,total_claim_amount,incident_month,incident_day,incident_dow,...,authorities_contacted_ambulance,policy_state_ca,policy_state_az,policy_state_nv,policy_state_id,policy_state_wa,policy_state_or,customer_gender_other,customer_gender_male,customer_gender_female
0,2,0,0,0,71600,8913.668763,80513.668763,3,17,6,...,0,0,0,0,0,1,0,0,0,0
1,3,4,0,1,6400,19746.724395,26146.724395,12,11,2,...,0,1,0,0,0,0,0,0,1,0
2,2,1,0,0,3400,27987.704652,31387.704652,5,8,2,...,0,1,0,0,0,0,0,0,0,1
3,3,1,2,1,18100,35945.149280,54045.149280,5,20,0,...,0,1,0,0,0,0,0,0,0,0
4,1,0,0,0,16100,8307.428880,24407.428880,5,1,2,...,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4004,2,0,3,0,15700,1494.409549,17194.409549,8,7,2,...,0,1,0,0,0,0,0,0,1,0
4005,2,0,1,0,34900,14837.909976,49737.909976,1,10,3,...,0,0,0,1,0,0,0,0,0,1
4006,1,0,0,0,11700,12421.253027,24121.253027,6,3,0,...,0,1,0,0,0,0,0,0,1,0
4007,3,0,2,1,14000,10991.604997,24991.604997,3,8,4,...,0,1,0,0,0,0,0,0,0,1


In [4]:
train_label

,fraud
0,0
1,0
2,0
3,0
4,0
...,...
4004,1
4005,0
4006,0
4007,0


In [5]:
dtrain = xgb.DMatrix(train_features, label=train_label)

ImportError: cannot import name 'Int64Index' from 'pandas' (/opt/conda/lib/python3.10/site-packages/pandas/__init__.py)

In [ ]:
# Cross-validate on training data
cv_results = xgb.cv(
    params=hyperparams,
    dtrain=dtrain,
    num_boost_round=num_boost_round,
    nfold=nfold,
    early_stopping_rounds=early_stopping_rounds,
    metrics=["auc"],
    seed=10,
)


metrics_data = {
    "binary_classification_metrics": {
        "validation:auc": {
            "value": cv_results.iloc[-1]["test-auc-mean"],
            "standard_deviation": cv_results.iloc[-1]["test-auc-std"]
        },
        "train:auc": {
            "value": cv_results.iloc[-1]["train-auc-mean"],
            "standard_deviation": cv_results.iloc[-1]["train-auc-std"]
        },
    }
}


print(f"Cross-validated train-auc:{cv_results.iloc[-1]['train-auc-mean']:.2f}")
print(f"Cross-validated validation-auc:{cv_results.iloc[-1]['test-auc-mean']:.2f}")